---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [4]:
import re
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [8]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    uni_file = open('university_towns.txt')
    uni_text = uni_file.read()
    uni_file.close()
    lines = uni_text.split('\n')
    state_pattern = "(.+)\[edit\]"
    region_pattern = "(.+)\("
    #state_pattern = "(\w+\s?\w+?)\[edit\]"
    #region_pattern = "(\w+\s?\w+?-?\w+)\s?\("
    resp = []
    for item in lines:
        match1 = re.search(state_pattern, item)
        if match1:
            state = match1.group(1)
        match2 = re.search(region_pattern, item)
        if match2:
            town = match2.group(1)
            town = town.strip()
            resp.append([state, town])
        if match2 == None and match1 == None and len(item) != 0:
            mytown = item.strip('\n')
            resp.append([state, mytown])
    cols = ['State', 'RegionName']
    answer1 = pd.DataFrame(resp, columns=cols)
    return answer1

totto = get_list_of_university_towns()

In [8]:
def get_recession_zeros(value):
    """
    if pib decreases return zero otherwise return one
    """
    if value < 0:
        return 0
    return 1

def helper_function123(pib_data):
    """
    creates pattern to detect recession start and end
    input: pib series column from year1 to year2
    ouput: a tuple with start and end index
    """
    #bruto = pd.DataFrame({'quarters': conty, 'pib': pib_data})
    bruto = pib_data
    #bruto = bruto.rename(columns={'GDP in billions of chained 2009 dollars': 'pib'})
    bruto.index = bruto.Quarterly.apply(pd.Period)
    bruto['Changes'] = bruto.pib.diff()
    bruto['Zeros'] = bruto.Changes.apply(get_recession_zeros)
    cad = bruto.Zeros.map(lambda x: str(x)).str.cat()
    inicio = "00"
    fin = "00.+?(11)"
    # cadspan shows the start and end of recession, it's a tuple
    cadspan = re.search(fin, cad).span()
    return cadspan

def get_gdp_data():
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp.iloc[219:, [4,6]]
    gdp.columns = ['Quarterly', 'GDP in billions of chained 2009 dollars']
    gdp = gdp.reset_index()
    del gdp['index']
    gdp = gdp.rename(columns={'GDP in billions of chained 2009 dollars': 'pib'})
    return gdp

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    mydataframe = get_gdp_data()
    recession_start = helper_function123(mydataframe)[0]
    start = mydataframe.iloc[recession_start, 0]
    return start

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    mydataframe = get_gdp_data()
    recession_end = helper_function123(mydataframe)[1]
    end = mydataframe.iloc[recession_end, 0]
    return end

In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    mydataframe = get_gdp_data()
    cadspan = helper_function123(mydataframe)
    recesion_min = mydataframe.loc[mydataframe.pib.iloc[cadspan[0]:cadspan[1]].argmin()].Quarterly
    return recesion_min

In [11]:
def create_quarters():
    """
    creates quarters names from year 2000
    """
    quarters = []
    for year in range(2000, 2017):
        for quarter in ['q1', 'q2', 'q3', 'q4']:
            elem = str(year) + quarter
            quarters.append(elem)
    return quarters

def processdf(cols):
    """
    converts monthly pib data to quarterly pib data, output is a dataframe
    """
    cont = {}
    fin = cols.shape[1]
    xcols = create_quarters()
    control = 0
    for i in range(0, fin, 3):
        try:
            prom = cols.iloc[:, i:i+3].mean(axis=1)
            cont[xcols[control]] = prom
            control += 1
        except IndexError:
            control -= 1
            prom = cols.iloc[:, i:].mean(axis=1)
            cont[xcols[control]] = prom
            #control += 1
            break
    dframe = pd.DataFrame(cont)
    return dframe

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    years_data = data.loc[:, '2000-01':]
    quarters_data = processdf(years_data)
    indices = data.loc[:, ['State', 'RegionName',]]
    indices['State'] = indices['State'].map(lambda x: states[x])
    resp = pd.concat([indices, quarters_data], axis=1)
    resp = resp.sort_values('State')
    resp = resp.set_index(['State', 'RegionName'])
    return resp

In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    p_objective = 0.01
    
    data = convert_housing_data_to_quarters()
    ul = get_list_of_university_towns()
    indexlist = list(map(tuple,ul.values))
    
    quarters_list = create_quarters()
    rec_start = get_recession_start()
    quarter_before_rec = quarters_list[quarters_list.index(rec_start) - 1]
    rec_bottom = get_recession_bottom()
    #data['Changes'] = (data['2009q2'] - data['2008q3']).apply(show_decline_or_growth)
    data['PriceRatio'] = data[quarter_before_rec].div(data[rec_bottom])
    uni_towns = data.loc[indexlist]
    non_uni_towns = data.loc[~data.index.isin(indexlist)]
    scipy_ttest = ttest_ind(uni_towns.PriceRatio, non_uni_towns.PriceRatio, nan_policy='omit')
    psvalue = scipy_ttest.pvalue
    different = psvalue < p_objective
    mean_price_ratio_univ = uni_towns.PriceRatio.mean()
    non_mean_price_ratio_univ = non_uni_towns.PriceRatio.mean()
    if mean_price_ratio_univ < non_mean_price_ratio_univ:
        better = 'university town'
    else:
        better = 'non-university town'
    return different, psvalue, better